In [1]:
from __future__ import print_function
import pandas as pd 
import sys
sys.path.append('/ref/pipelines/')
import kang
import subprocess
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
file_kaks = './Ah2Ah.collinearity.kaks'

df_kaks   = pd.read_csv(file_kaks,sep='\t',comment='#',header=None)
df_kaks['synteny id'] = df_kaks[0].apply(lambda x : x.split('-')[0].strip())

In [4]:
df_gff = pd.read_csv('./Ah2Ah.gff',sep='\t',header=None)

In [5]:
df_gff_ix = df_gff.set_index(1)

In [7]:
df_kaks['posA_l'] = df_kaks[1].apply(lambda x : df_gff_ix.xs(x)[2])
df_kaks['posB_l'] = df_kaks[2].apply(lambda x : df_gff_ix.xs(x)[2])
df_kaks['posA_r'] = df_kaks[1].apply(lambda x : df_gff_ix.xs(x)[3])
df_kaks['posB_r'] = df_kaks[2].apply(lambda x : df_gff_ix.xs(x)[3])
df_kaks['chrA']   = df_kaks[1].apply(lambda x : df_gff_ix.xs(x)[0])
df_kaks['chrB']   = df_kaks[2].apply(lambda x : df_gff_ix.xs(x)[0])

In [8]:
df_kaks_ix = df_kaks.set_index('synteny id')

In [11]:
file_fa = '/ref/References/Aha/Ahal.assembly.scf.fasta'
dic_fa  = kang.Fasta2dic(file_fa)

In [12]:
def save_seq(name,seq):
    outfile = 'temp.%s.fa'%name
    with open(outfile,'w') as f:
        print('>%s'%name,file=f)
        print(seq,file=f)
    return outfile

In [116]:
def get_two_blast_from_synteny(selected_sbix):
    #selected_sbix = '336'
    edf           = df_kaks_ix.xs(selected_sbix)
    echrA          = edf['chrA'].values[0]
    echrB          = edf['chrB'].values[0]
    eposA_from    = np.min(edf['posA_l'])
    eposA_to      = np.max(edf['posA_r'])
    eposB_from    = np.min(edf['posB_l'])
    eposB_to      = np.max(edf['posB_r'])

    seqA = dic_fa[echrA][eposA_from:eposA_to]
    seqB = dic_fa[echrB][eposB_from:eposB_to]

    seqA_name=save_seq('%s_a'%selected_sbix,seqA)
    seqB_name=save_seq('%s_b'%selected_sbix,seqB)

    outxml = 'temp.%s.xml'%selected_sbix
    print('cmd: blastn -query %s -subject %s -outfmt 5 -out %s'%(seqA_name,seqB_name,outxml))
    subprocess.call('blastn -query %s -subject %s -outfmt 5 -out %s'%(seqA_name,seqB_name,outxml),shell=True)

    tree = ET.parse(outxml)
    root = tree.getroot()
    dic = {'hitfrom':[],
           'hitto':[]
          }
    with open('%s.twoblast.parsed.txt'%selected_sbix,'w') as f:
        print ('# synteny block id : %s'%selected_sbix,file=f)
        aligned_pos_ref = {0}
        aligned_pos_query = {0}
        for hsps in root.iter('Hsp'):
            hsp_num      = hsps.find('Hsp_num').text
            hsp_hit_from = hsps.find('Hsp_hit-from').text
            hsp_hit_to   = hsps.find('Hsp_hit-to').text
            hsp_query_from = hsps.find('Hsp_query-from').text
            hsp_query_to   = hsps.find('Hsp_query-to').text
            hsp_align_len  = hsps.find('Hsp_align-len').text
            dic['hitfrom'].append(hsp_hit_from)
            dic['hitto'].append(hsp_hit_to)
            aligned_pos_ref   = aligned_pos_ref | set(range(int(hsp_hit_from),int(hsp_hit_to)))
            aligned_pos_query = aligned_pos_query | set(range(int(hsp_query_from),int(hsp_query_to)))
            print ('## alignment id: %s'%hsp_num,file=f)
            print ('## alignment position query (%s) from %d to %d'%(echrA,eposA_from+int(hsp_hit_from),eposA_from+int(hsp_hit_to)),file=f)
            print ('## alignment position ref (%s) from %d to %d'%(echrB,eposB_from+int(hsp_query_from),eposB_from+int(hsp_query_to)),file=f)
            print ('## alignment length : %s bp'%hsp_align_len,file=f)
            hsp_qseq = hsps.find('Hsp_qseq').text
            hsp_hseq = hsps.find('Hsp_hseq').text

            kang.seq_comp(['%s_%s'%(selected_sbix,echrB),'%s_%s'%(selected_sbix,echrA)],[hsp_hseq,hsp_qseq],f)
        aligned_pos_ref      = aligned_pos_ref  - {0}
        aligned_pos_query    = aligned_pos_query- {0}
        aligned_length_ref   = len(aligned_pos_ref)
        aligned_length_query = len(aligned_pos_query)
        print('# ref   : %s:%s..%s (%d bp)'%(echrB,eposB_from,eposB_to,len(seqB)),file=f)
        print('# query : %s:%s..%s (%d bp)'%(echrA,eposA_from,eposA_to,len(seqA)),file=f)
        print('# ref_scaffold   : %s (%d bp)'%(echrB,len(dic_fa[echrB])),file=f)
        print('# query_scaffold : %s (%d bp)'%(echrA,len(dic_fa[echrA])),file=f)
        print('# total aligned length from ref : %d'%aligned_length_ref,file=f)
        print('# total aligned length from query : %d'%aligned_length_query,file=f)
        print('# ratio of aligned length to ref_length : %f'%(float(aligned_length_ref)/len(seqB)),file=f)
        print('# ratio of aligned length to query_length : %f'%(float(aligned_length_query)/len(seqA)),file=f)
        print('# ratio of aligned length to ref_scaffold_length : %f'%(float(aligned_length_ref)/len(dic_fa[echrB])),file=f)
        print('# ratio of aligned length to query_scaffold_length : %f'%(float(aligned_length_query)/len(dic_fa[echrA])),file=f)
    #subprocess.call('rm temp.*',shell=True)

In [163]:
def get_two_blast_from_two_scaffold(echrA,echrB):


    seqA = dic_fa[echrA]
    seqB = dic_fa[echrB]

    seqA_name=save_seq('%s'%echrA,seqA)
    seqB_name=save_seq('%s'%echrB,seqB)

    outxml = 'temp.%s.%s.xml'%(echrA,echrB)
    print('cmd: blastn -query %s -subject %s -outfmt 5 -out %s'%(seqA_name,seqB_name,outxml))
    subprocess.call('blastn -query %s -subject %s -outfmt 5 -out %s'%(seqA_name,seqB_name,outxml),shell=True)

    tree = ET.parse(outxml)
    root = tree.getroot()
    dic = {'hitfrom':[],
           'hitto':[]
          }
    with open('%s.twoblast.parsed.txt'%(echrA+'_'+echrB),'w') as f:
        print ('# %s and %s twoblast comparision'%(echrA,echrB),file=f)
        aligned_pos_ref = {0}
        aligned_pos_query = {0}
        for hsps in root.iter('Hsp'):
            hsp_num      = hsps.find('Hsp_num').text
            hsp_hit_from = hsps.find('Hsp_hit-from').text
            hsp_hit_to   = hsps.find('Hsp_hit-to').text
            hsp_query_from = hsps.find('Hsp_query-from').text
            hsp_query_to   = hsps.find('Hsp_query-to').text
            hsp_align_len  = hsps.find('Hsp_align-len').text
            hsp_hit_frame  = hsps.find('Hsp_hit-frame').text
            hsp_query_frame = hsps.find('Hsp_query-frame').text
            
            if hsp_hit_frame == '-1':
                hsp_hit_from, hsp_hit_to = hsp_hit_to, hsp_hit_from
            if hsp_query_frame == '-1':
                hsp_query_from, hsp_query_to = hsp_query_to, hsp_query_from
            
            
            # redundunt alignment skip
            if len(aligned_pos_ref & set(range(int(hsp_hit_from),int(hsp_hit_to))))>100:
                continue
            
            aligned_pos_ref   = aligned_pos_ref | set(range(int(hsp_hit_from),int(hsp_hit_to)))
            aligned_pos_query = aligned_pos_query | set(range(int(hsp_query_from),int(hsp_query_to)))
            
            print ('## alignment id: %s'%hsp_num,file=f)
            print ('## alignment position query (%s) from %d to %d'%(echrA,int(hsp_query_from),int(hsp_query_to)),file=f)
            print ('## alignment position ref (%s) from %d to %d'%(echrB,int(hsp_hit_from),int(hsp_hit_to)),file=f)
            print ('## alignment length : %s bp'%hsp_align_len,file=f)
            hsp_qseq = hsps.find('Hsp_qseq').text
            hsp_hseq = hsps.find('Hsp_hseq').text

            kang.seq_comp(['%s'%(echrB),'%s'%(echrA)],[hsp_hseq,hsp_qseq],f)
        aligned_pos_ref      = aligned_pos_ref  - {0}
        aligned_pos_query    = aligned_pos_query- {0}
        aligned_length_ref   = len(aligned_pos_ref)
        aligned_length_query = len(aligned_pos_query)
        
        print('# ref_scaffold   : %s (%d bp)'%(echrB,len(dic_fa[echrB])),file=f)
        print('# query_scaffold : %s (%d bp)'%(echrA,len(dic_fa[echrA])),file=f)
        print('# total aligned length from ref : %d'%aligned_length_ref,file=f)
        print('# total aligned length from query : %d'%aligned_length_query,file=f)
        print('# ratio of aligned length to ref_length : %f'%(float(aligned_length_ref)/len(seqB)),file=f)
        print('# ratio of aligned length to query_length : %f'%(float(aligned_length_query)/len(seqA)),file=f)

    subprocess.call('rm temp.*',shell=True)
    return float(aligned_length_query)/len(seqA), float(aligned_length_ref)/len(seqB)

In [156]:
get_two_blast_from_synteny('336')

cmd: blastn -query temp.336_a.fa -subject temp.336_b.fa -outfmt 5 -out temp.336.xml


In [125]:
get_two_blast_from_synteny('337')

cmd: blastn -query temp.337_a.fa -subject temp.337_b.fa -outfmt 5 -out temp.337.xml


In [126]:
get_two_blast_from_synteny('412')

cmd: blastn -query temp.412_a.fa -subject temp.412_b.fa -outfmt 5 -out temp.412.xml


In [160]:
get_two_blast_from_two_scaffold('scaffold-935','scaffold-36')

cmd: blastn -query temp.scaffold-935.fa -subject temp.scaffold-36.fa -outfmt 5 -out temp.scaffold-935.scaffold-36.xml


(0.9402726639466613, 0.01123730255997835)

In [148]:
df_lowks_sb = pd.read_pickle('./df_lowks_sb.pk')

In [162]:
df_lowks_sb['ChrA_twoblast_aligned_fraction'] = 0. 
df_lowks_sb['ChrB_twoblast_aligned_fraction'] = 0.
for ix in set(df_lowks_sb.index):
    edf = df_lowks_sb.xs(ix)
    chrA = edf['ChrA']
    chrB = edf['ChrB']
    chrA_two_blast_aligned_ratio, chrB_two_blast_aligned_ratio = get_two_blast_from_two_scaffold(chrA,chrB)
    df_lowks_sb = df_lowks_sb.set_value(ix,'ChrA_twoblast_aligned_fraction',chrA_two_blast_aligned_ratio)
    df_lowks_sb = df_lowks_sb.set_value(ix,'ChrB_twoblast_aligned_fraction',chrB_two_blast_aligned_ratio)

cmd: blastn -query temp.scaffold-80.fa -subject temp.scaffold-902.fa -outfmt 5 -out temp.scaffold-80.scaffold-902.xml
cmd: blastn -query temp.scaffold-77.fa -subject temp.scaffold-947.fa -outfmt 5 -out temp.scaffold-77.scaffold-947.xml
cmd: blastn -query temp.scaffold-77.fa -subject temp.scaffold-942.fa -outfmt 5 -out temp.scaffold-77.scaffold-942.xml
cmd: blastn -query temp.001501F-pilon.fa -subject temp.001931F-pilon.fa -outfmt 5 -out temp.001501F-pilon.001931F-pilon.xml
cmd: blastn -query temp.scaffold-36.fa -subject temp.scaffold-918.fa -outfmt 5 -out temp.scaffold-36.scaffold-918.xml
cmd: blastn -query temp.scaffold-36.fa -subject temp.scaffold-935.fa -outfmt 5 -out temp.scaffold-36.scaffold-935.xml
cmd: blastn -query temp.scaffold-104.fa -subject temp.scaffold-956.fa -outfmt 5 -out temp.scaffold-104.scaffold-956.xml
cmd: blastn -query temp.scaffold-145.fa -subject temp.scaffold-177.fa -outfmt 5 -out temp.scaffold-145.scaffold-177.xml
cmd: blastn -query temp.scaffold-140.fa -subje

In [151]:
#df_lowks_sb.set_value(ix,'ChrA_twoblast_aligned_fraction',chrA_two_blast_aligned_ratio)
df_lowks_sb.to_excel('lowKs_scaffolds.xls')

In [150]:
df_lowks_sb.head()

,ChrA,ChrA_len,ChrB,ChrB_len,Ks,ratioA,ratioB,synblock id,synblockA_len,synblockB_len,ChrA_twoblast_aligned_fraction,ChrB_twoblast_aligned_fraction
0,scaffold-80,1910584,scaffold-902,32793,0.146745,0.036198,0.931449,449,69160,30545,0.018346,0.649651
1,scaffold-77,3518929,scaffold-947,56809,0.040018,0.012773,0.763770,443,44949,43389,0.013912,0.028869
2,scaffold-77,3518929,scaffold-942,54235,0.058247,0.011934,0.809588,442,41995,43908,0.013582,0.035401
3,001501F-pilon,27400,001931F-pilon,15227,0.055767,0.392883,0.736061,0,10765,11208,0.505255,0.905760
4,scaffold-36,4677813,scaffold-918,41062,0.038916,0.007073,0.859529,335,33086,35294,0.009713,0.944888


In [141]:
chrA_two_blast_aligned_ratio

0.7402350307778399

In [144]:
ix

37